In [466]:
%reload_ext autoreload

In [1]:
%load_ext autoreload
%aimport robit
%aimport ik
%autoreload 1

In [2]:
import robit
import ik

In [346]:
import matplotlib.pyplot as plt
import numpy as np
import time
import odrive
from odrive.utils import *
import tinyik
import math

In [23]:
R = robit.Robit([('205B39964D4D', 'Test1', 'Test2')])

looking for device 205B39964D4D
found device


In [29]:
print (R.devices['205B39964D4D'].axis0.controller.config)

control_mode = 3 (int)
pos_gain = 50.0 (float)
vel_gain = 5.999999848427251e-05 (float)
vel_integrator_gain = 0.0 (float)
vel_limit = 40000.0 (float)
vel_limit_tolerance = 1.2000000476837158 (float)
vel_ramp_rate = 10000.0 (float)
setpoints_in_cpr = False (bool)


In [430]:
R.set_state ('Test1', 'CONTROL')
R.set_state ('Test2', 'CONTROL')

In [30]:
R.set_state ('Test1', 'CALIBRATE')
R.set_state ('Test2', 'CALIBRATE')

In [291]:
R.set_state ('Test1', 'IDLE')
R.set_state ('Test2', 'IDLE')

In [188]:
R.reset(['Test1', 'Test2'], [0, 0])

In [260]:
R.snap(['Test1'],[-1.141])

In [261]:
R.snap(['Test2'],[-1.141])

In [262]:
R.snap(['Test1'],[1.141])

In [263]:
R.snap(['Test2'],[1.141])

In [264]:
R.snap(['Test1'],[0])

In [265]:
R.snap(['Test2'],[0])

In [74]:
R.snap(['Test1', 'Test2'], [.5, .5])

In [75]:
R.snap(['Test1', 'Test2'], [-.5, -.5])

In [293]:
R.snap(['Test1', 'Test2'], [0., 0.])

In [71]:
R.test_sin (['Test1', 'Test2'], 2, 1)

In [252]:
for serial in R.devices.keys ():
    print (serial)
    dump_errors (R.devices[serial], True)

205B39964D4D
axis0
  axis: no error
  motor: no error
  encoder: no error
  controller: no error
axis1
  axis: no error
  motor: no error
  encoder: no error
  controller: no error


In [298]:
# Front left
leg = tinyik.Actuator(['y', [0., 0., -116.], 'y', [0., 0., -125.]])



In [299]:
deg = [0.0001, 0.0001]
leg.angles = np.deg2rad(deg)
np.round(leg.ee)

array([  -0.,    0., -241.])

In [363]:
i = 0

In [427]:
size = 80
x,z = math.cos (i) * size * 1.5, -math.sin(i) * size
i += .5
x, z

(-57.064431359519105, 70.3756607977336)

In [428]:
center = [ 0.,    0., -120.]

leg.ee = [center[0] + x, center[1], center[2] + z]
angles = np.clip(leg.angles, -3, 3)
angles[1] = -angles[1]
angles

array([-0.51203143, -2.50746494])

In [429]:
R.snap(['Test2', 'Test1'], angles)

In [440]:
start = time.time ()
ik = 0
for i in range (0, 120, 1):
    size = 60
    x,z = math.cos (i/10) * size * 1.5, -math.sin(i/10) * size

    center = [ 0.,    0., -140.]
    
    
    point = [center[0] + x, center[1], center[2] + z]
    current = time.time ()
    leg.ee = point
    ik += time.time () - current
    angles = np.clip(leg.angles, -3, 3)
    
    angles[1] = -angles[1]
    angles

    R.snap(['Test2', 'Test1'], angles)
    
#    time.sleep (.001)

print (ik)
print (time.time() - start)

12.290964841842651
12.430148839950562


In [441]:
R.set_state ('Test1', 'IDLE')
R.set_state ('Test2', 'IDLE')

In [255]:
for label in R.motor_table.keys ():
    motor = R.get_motor (label)
    motor.motor.config.current_lim = 20
    motor.controller.config.vel_limit = 25000
    motor.motor.config.calibration_current = 5
    motor.motor.config.pole_pairs = 12
    motor.motor.config.motor_type = 0
    motor.encoder.config.cpr = 8192
    
    motor.controller.config.control_mode = 3
    motor.controller.config.pos_gain = 50
    motor.controller.config.vel_gain = 6 / 100000
    motor.controller.config.vel_integrator_gain = 0
    motor.controller.config.vel_limit_tolerance = 1.2
    motor.controller.config.vel_ramp_rate = 10000

In [ ]:
for device in R.devices.values ():
    device.save_configuration()
    device.reboot()

In [117]:
import ik

In [384]:
angles = ik.solve ([20, 0, -100], True)
angles

20.0 115.0 115.0
101.9803902718557 20.0
height: 100.0
-0.6219281663516069
0.0
-1.120999466370539


(0.0, 1.120999466370539, -2.241998932741078)